In [ ]:
"""
1.语音信号读写,语音信号的基本特征
2.抽取内插滤波
3.信号加噪和信噪比统计
::录制一段语音，读取、画出信号波形和功率谱
::自学：语音信号写入文件
::根据抽样率从高倍采集得到的信号中抽取
::抗混叠滤波
::::方法1: 用APP构造滤波器
::::::在python中导入.mat文件
::::::用scipy.io.loadmat()函数读取.mat文件
或者
::::::用scipy.signal.firwin()函数设计滤波器
::::::用scipy.signal.lfilter()函数实现滤波
"""

In [ ]:
# 导入库
import numpy as np
import matplotlib.pyplot as plt
import scipy.io.wavfile as wav
import scipy.signal as signal
import scipy.io as io

In [ ]:
# 读取、画出信号波形和功率谱
# 读取语音信号
# win下读取wav文件
# rate, data = wav.read('C:\\Users\\88486\\Desktop\\WorkSpace\\Up&Load\\通信原理\\suno.wav')

# linux下读取wav文件
rate, data = wav.read('/home/yfc/文档/GitHub/Up-Load/通信原理/suno.wav')

# 画出信号波形
plt.figure()
plt.plot(data)
plt.title('voice wave')
plt.show()

# 计算功率谱
"""
f:返回的频率数组;
Pxx:对应每个频率点的功率谱密度值。
"""
f, Pxx = signal.periodogram(data, rate)
# 公式计算功率谱:
data_FFT=np.fft.fft(data)
Pf=abs(data_FFT)**2/len(data)
f=np.linspace(0,rate,len(data))

# 画出功率谱
plt.figure()
plt.plot(f, Pxx)
plt.title('voice psd')
plt.show()


抗混叠滤波（Anti-Aliasing Filter）是信号处理中的一种低通滤波器，用于限制信号的带宽，防止高频信号混叠到低频带中。在数字信号处理中，特别是在模拟信号转换为数字信号的过程中，抗混叠滤波器是非常关键的。

混叠现象发生在采样过程中，当模拟信号包含频率高于采样频率一半（奈奎斯特频率）的分量时，这些高频分量在数字化的过程中会被折叠到低频范围内，从而导致信号失真。抗混叠滤波器的作用就是在采样之前过滤掉这些高于奈奎斯特频率的信号分量，确保采样后的数字信号能够忠实地表示原始的模拟信号。

例如，如果我们要对一个音频信号进行数字化，采样频率可能是44.1 kHz，这意味着所有超过22.05 kHz的频率分量都需要被过滤掉，以避免混叠现象。抗混叠滤波器通常设计为在截止频率（即采样频率的一半）处提供足够的衰减，以确保混叠分量不会对信号质量产生显著影响。

在图像处理中，抗混叠滤波器也用于减少或消除图像中的锯齿效果（aliasing），特别是在图像放大或从栅格图像生成矢量图像时。通过在采样之前对图像进行模糊处理，可以减少高频分量的影响，从而减少混叠现象。

In [ ]:

# 抗混叠滤波的英文是anti-aliasing filter
from scipy import signal
import numpy as np
import matplotlib.pyplot as plt
# Butterworth低通滤波器设计
def design_lowpass_filter(cutoff_frequency, sample_rate, filter_order):
    nyquist_rate = sample_rate / 2.0
    normal_cutoff = cutoff_frequency / nyquist_rate
    b, a = signal.butter(filter_order, normal_cutoff, btype='low', analog=False)
    return b, a
# 应用滤波器
def apply_lowpass_filter(data, cutoff_frequency, sample_rate, filter_order):
    b, a = design_lowpass_filter(cutoff_frequency, sample_rate, filter_order)
    filtered_data = signal.filtfilt(b, a, data)
    return filtered_data


In [ ]:
# 内插与抽取可以用scipy库中的interpolate模块实现
import numpy as np
from scipy import interpolate
# 生成原始数据
x = np.linspace(0, 4, 13)
y = np.array([0, 0.8415, 0.9093, 0.1411, -0.7568, -0.9589, -0.2794, 0.657, 0.9894, 0.4121, -0.544, -0.9999, -0.5366])
# 生成插值函数
f = interpolate.interp1d(x, y)
# 生成插值点
x_new = np.linspace(0, 4, 30)
y_new = f(x_new)
# 画出原始数据和插值数据
import matplotlib.pyplot as plt
plt.plot(x, y, 'o', x_new, y_new, '-')
plt.show()

# 信号采样
import numpy as np
# 生成原始数据
data = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
rate = 10
new_rate = 5
# 信号采样
data_new = data[::2]
print(data_new)

# 内插函数
def interp(x, y, x_new):#x,y为原始数据，x_new为新的插值点
    y_new = np.interp(x_new, x, y)
    return y_new #返回新的插值点

# 对信号采样
def downsample(data, rate, new_rate): #data为原始数据，rate为原始采样率，new_rate为新的采样率
    # 采样间隔
    interval = int(rate / new_rate)
    # 采样
    data_new = data[::interval]
    return data_new #返回新的采样数据
# 内插和抽样的英文是interpolation和downsampling
# 实现内插，输入为原始数据，原始采样率，新的采样率
def interp(data, rate, new_rate):
    # 生成原始数据的时间序列
    time = np.linspace(0, len(data) / rate, len(data))
    # 生成新的时间序列
    new_time = np.linspace(0, len(data) / rate, int(len(data) * new_rate / rate))
    # 生成插值函数
    f = interpolate.interp1d(time, data)
    # 生成插值数据
    data_new = f(new_time)
    return data_new #返回新的插值数据

# 实现抽取，输入为原始数据，原始采样率，新的采样率
def downsample(data, rate, new_rate):
    # 生成原始数据的时间序列
    time = np.linspace(0, len(data) / rate, len(data))
    # 生成新的时间序列
    new_time = np.linspace(0, len(data) / rate, int(len(data) * new_rate / rate))
    # 生成插值函数
    f = interpolate.interp1d(time, data)
    # 生成插值数据
    data_new = f(new_time)
    # 生成抽取数据
    data_downsample = data_new[::int(rate / new_rate)]
    return data_downsample #返回新的抽取数据

In [ ]:
# python读入matlab的.mat文件
# import scipy.io as io
# data = io.loadmat('data.mat')
# print(data)
# 用scipy.signal.firwin()函数设计滤波器
# 用scipy.signal.lfilter()函数实现滤波
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as signal
# 生成原始数据
data = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
rate = 10
# 设计滤波器
b = signal.firwin(3, 0.5)
# 实现滤波
data_filtered = signal.lfilter(b, 1, data)
# 画出原始数据和滤波数据
plt.plot(data, label='data')
plt.plot(data_filtered, label='data_filtered')
plt.legend()
plt.show()
print(data)
print(data_filtered)

